# Classificador

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
pd.options.display.max_rows = 13

Limpeza simples:

In [2]:
import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    import string
    punctuation = '[\/!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

## Análise dos tweets marca Xiaomi

A análise foi baseada na classificação dos tweets, de forma binária, sendo favoráveis a marca (0), ou contra a marca (1)

In [3]:
import os

os.getcwd()

'C:\\Users\\User\\Documents\\Disciplinas\\2° semestre\\Ciência dos Dados\\P2_tweets'

In [4]:
tweets = pd.read_excel('xiaomi_1.xlsx')

In [5]:
tweets;

In [6]:
#apagar tweets que foram repetidos ('RTs')
n_repete_tt = tweets.drop_duplicates()
n_repete_tt;

In [7]:
dict_n_rep_tt= n_repete_tt.to_dict('list')

In [8]:
dict_n_rep_tt

{'Treinamento': ['@marcusc13 @xiaomibrasil @apple @motorolabr @samsungbrasil na verdade os celulares da samsung são de boa qualidade, o que faz os aparelhos apresentarem lentidão com um tempo de uso é a interface implementada no sistema. ela é pesada, lenta e com várias falhas. a xiaomi por exemplo, usa a miui, uma interface madura e bem otimizada.',
  'esse cel xiaomi é excelente! 😁',
  '@hugorambo18 não temos vagas, abandonou a família xiaomi',
  '@alysonhoffmann comprei 2 fones da xiaomi no dia dos pais, custavam quase 200 cada, no brasil, paguei 85 em cada com frete grátis e chegou em 15 dias em perfeito estado e sem taxas.',
  '@itsramss aqui e team xiaomi ramenex junte-se a nós',
  'xiaomi eh a o anti cristo não caiam nessa https://t.co/ctkzhk3wlq',
  'mano, nem tô acreditando que depois de indicar pra todo mundo, eu finalmente virei testemunha de xiaomi 😂😂😂  inclusive, tô amando meu rn7 @xiaomibrasil. continuarei propagando ao mundo teus feitos 😂😂😂♥️',
  '@grrvide1 me sinto uma 

In [9]:
lista_tt = dict_n_rep_tt['Treinamento']
lista_tt

['@marcusc13 @xiaomibrasil @apple @motorolabr @samsungbrasil na verdade os celulares da samsung são de boa qualidade, o que faz os aparelhos apresentarem lentidão com um tempo de uso é a interface implementada no sistema. ela é pesada, lenta e com várias falhas. a xiaomi por exemplo, usa a miui, uma interface madura e bem otimizada.',
 'esse cel xiaomi é excelente! 😁',
 '@hugorambo18 não temos vagas, abandonou a família xiaomi',
 '@alysonhoffmann comprei 2 fones da xiaomi no dia dos pais, custavam quase 200 cada, no brasil, paguei 85 em cada com frete grátis e chegou em 15 dias em perfeito estado e sem taxas.',
 '@itsramss aqui e team xiaomi ramenex junte-se a nós',
 'xiaomi eh a o anti cristo não caiam nessa https://t.co/ctkzhk3wlq',
 'mano, nem tô acreditando que depois de indicar pra todo mundo, eu finalmente virei testemunha de xiaomi 😂😂😂  inclusive, tô amando meu rn7 @xiaomibrasil. continuarei propagando ao mundo teus feitos 😂😂😂♥️',
 '@grrvide1 me sinto uma testemunha do xiaomi ok

In [10]:
lista_classificacao = dict_n_rep_tt['Classificação']

In [11]:
i=0
lista_sem_caracteres = []
for i in range(len(lista_tt)):
    novo_tt = cleanup(lista_tt[i])
    lista_sem_caracteres.append(novo_tt.lower())
    i += 1

In [12]:
lista_sem_caracteres

['@marcusc13 @xiaomibrasil @apple @motorolabr @samsungbrasil na verdade os celulares da samsung são de boa qualidade  o que faz os aparelhos apresentarem lentidão com um tempo de uso é a interface implementada no sistema  ela é pesada  lenta e com várias falhas  a xiaomi por exemplo  usa a miui  uma interface madura e bem otimizada ',
 'esse cel xiaomi é excelente  😁',
 '@hugorambo18 não temos vagas  abandonou a família xiaomi',
 '@alysonhoffmann comprei 2 fones da xiaomi no dia dos pais  custavam quase 200 cada  no brasil  paguei 85 em cada com frete grátis e chegou em 15 dias em perfeito estado e sem taxas ',
 '@itsramss aqui e team xiaomi ramenex junte se a nós',
 'xiaomi eh a o anti cristo não caiam nessa https   t co ctkzhk3wlq',
 'mano  nem tô acreditando que depois de indicar pra todo mundo  eu finalmente virei testemunha de xiaomi 😂😂😂  inclusive  tô amando meu rn7 @xiaomibrasil  continuarei propagando ao mundo teus feitos 😂😂😂♥️',
 '@grrvide1 me sinto uma testemunha do xiaomi ok

In [13]:
dict_df = {}
    
dict_df['Treinamento'] = lista_sem_caracteres
dict_df['Classificação'] = lista_classificacao

In [14]:
dict_df

{'Treinamento': ['@marcusc13 @xiaomibrasil @apple @motorolabr @samsungbrasil na verdade os celulares da samsung são de boa qualidade  o que faz os aparelhos apresentarem lentidão com um tempo de uso é a interface implementada no sistema  ela é pesada  lenta e com várias falhas  a xiaomi por exemplo  usa a miui  uma interface madura e bem otimizada ',
  'esse cel xiaomi é excelente  😁',
  '@hugorambo18 não temos vagas  abandonou a família xiaomi',
  '@alysonhoffmann comprei 2 fones da xiaomi no dia dos pais  custavam quase 200 cada  no brasil  paguei 85 em cada com frete grátis e chegou em 15 dias em perfeito estado e sem taxas ',
  '@itsramss aqui e team xiaomi ramenex junte se a nós',
  'xiaomi eh a o anti cristo não caiam nessa https   t co ctkzhk3wlq',
  'mano  nem tô acreditando que depois de indicar pra todo mundo  eu finalmente virei testemunha de xiaomi 😂😂😂  inclusive  tô amando meu rn7 @xiaomibrasil  continuarei propagando ao mundo teus feitos 😂😂😂♥️',
  '@grrvide1 me sinto uma 

In [15]:
#novo banco de dados apos limpeza simples dos tweets
novo_tweets = pd.DataFrame(data=dict_df)
novo_tweets

,Treinamento,Classificação
0,@marcusc13 @xiaomibrasil @apple @motorolabr @s...,0
1,esse cel xiaomi é excelente 😁,0
2,@hugorambo18 não temos vagas abandonou a famí...,1
3,@alysonhoffmann comprei 2 fones da xiaomi no d...,0
4,@itsramss aqui e team xiaomi ramenex junte se ...,0
5,xiaomi eh a o anti cristo não caiam nessa http...,1
...,...,...
321,fui pesquisar sobre a xiaomi e gostei sla ache...,0
322,isso demonstra que muitos equipamentos são de ...,0
323,@mumudesp se quiser eu pego meus slides e te a...,0


In [16]:
lista_sem_caracteres = dict_n_rep_tt['Treinamento']

Tweest a favor da marca:

In [17]:
afavor_tweets= novo_tweets.loc[(novo_tweets.Classificação==0)]

In [18]:
afavor_tweets

,Treinamento,Classificação
0,@marcusc13 @xiaomibrasil @apple @motorolabr @s...,0
1,esse cel xiaomi é excelente 😁,0
3,@alysonhoffmann comprei 2 fones da xiaomi no d...,0
4,@itsramss aqui e team xiaomi ramenex junte se ...,0
6,mano nem tô acreditando que depois de indicar...,0
7,@grrvide1 me sinto uma testemunha do xiaomi ok...,0
...,...,...
321,fui pesquisar sobre a xiaomi e gostei sla ache...,0
322,isso demonstra que muitos equipamentos são de ...,0
323,@mumudesp se quiser eu pego meus slides e te a...,0


In [19]:
n_afavor_tweets= novo_tweets.loc[(novo_tweets.Classificação==1)]

In [20]:
n_afavor_tweets

,Treinamento,Classificação
2,@hugorambo18 não temos vagas abandonou a famí...,1
5,xiaomi eh a o anti cristo não caiam nessa http...,1
12,rt @treviagabriel amiga n beije este homem el...,1
16,crossfiteiro é mais chato que fã da xiaomi,1
17,vou comprar meu xiaomi se eu tiver chorando é ...,1
19,será que esses xiaomi são bons mesmo,1
...,...,...
291,@guidofraga @matosrs_ tem um vídeo do loop inf...,1
301,e um fone xiaomi tb pq esse inferno de celular...,1
303,xiaomi lovers é quase uma religião ficam tenta...,1


In [35]:
n_afavor_tweets.columns
df = 

AttributeError: 'Index' object has no attribute 'count'

In [32]:
P_1 = n_afavor_tweets.count()/ (n_afavor_tweets.count() + afavor_tweets.count())
P_0 = 1-P_1

In [33]:
#Probabilidade de gostarem do produto Xiaomi; p(0)= 0,74
P_0

Treinamento      0.740061
Classificação    0.740061
dtype: float64

In [34]:
# Probabilidade de não gostarem do produto Xiaomi; p(1)= 0,25
P_1

Treinamento      0.259939
Classificação    0.259939
dtype: float64

In [21]:
#serie_afavor= pd.Series(afavor_tweets['Treinamento'])

In [22]:
#serie_afavor

In [23]:
#afavor_frel= [serie_afavor.spilt()].prod()

In [24]:
txt_afavor=''
for t in afavor_tweets['Treinamento']:
    t=str(t)
    txt_afavor+=t

In [37]:
afavor=pd.Series(txt_afavor.split())


In [38]:
afavor

0           @marcusc13
1        @xiaomibrasil
2               @apple
3          @motorolabr
4       @samsungbrasil
5                   na
             ...      
4167      condicionado
4168                da
4169              sala
4170               com
4171               ele
4172               🤩🤩😂
Length: 4173, dtype: object

In [27]:
tabela_afavor = afavor.value_counts()

In [28]:
tabela_afavor_relativa = afavor.value_counts(True)

In [29]:
tabela_afavor_relativa

xiaomi                   0.048167
o                        0.023005
que                      0.020848
de                       0.020848
e                        0.019650
um                       0.018452
                           ...   
bixin                    0.000240
finalmente               0.000240
esquecer                 0.000240
garanto                  0.000240
sigo                     0.000240
u44vghowmz@rodragon__    0.000240
Length: 1462, dtype: float64

In [30]:
txt_contra=''
for e in n_afavor_tweets['Treinamento']:
    e=str(e)
    txt_contra+=e

In [31]:
contra=pd.Series(txt_contra.split())

In [32]:
contra

0        @hugorambo18
1                 não
2               temos
3               vagas
4           abandonou
5                   a
            ...      
1039    @xiaomibrasil
1040          @xiaomi
1041             🙄🙄🙄🙄
1042           iphone
1043           50000x
1044           melhor
Length: 1045, dtype: object

In [33]:
tabela_contra_relativa = contra.value_counts(True)

In [45]:
tabela_contra_relativa

xiaomi                  0.067943
de                      0.023923
a                       0.022967
que                     0.022010
um                      0.021053
e                       0.018182
                          ...   
teu                     0.000957
@rslonik                0.000957
teve                    0.000957
jales                   0.000957
28zbm0ecbw@jamilrxrx    0.000957
comprando               0.000957
Length: 531, dtype: float64

In [39]:
twitter = afavor + contra

In [44]:
twitter

0        @marcusc13@hugorambo18
1              @xiaomibrasilnão
2                   @appletemos
3              @motorolabrvagas
4       @samsungbrasilabandonou
5                           naa
                 ...           
4167                        NaN
4168                        NaN
4169                        NaN
4170                        NaN
4171                        NaN
4172                        NaN
Length: 4173, dtype: object

In [43]:
serie_twitter = pd.Series(twitter.split())

AttributeError: 'Series' object has no attribute 'split'